In [ ]:
import os
os.chdir('/home/mohanty/VSCODE_PROJECTS/Medical-Detection3d-Toolkit/')

import pandas as pd
import SimpleITK as sitk
import shutil
import numpy as np

import json


In [3]:
from detection3d.scripts.gen_landmark_mask import gen_single_landmark_mask

# GET Data from databse

In [13]:
img_volume_path = '/home/mohanty/Documents/ForTrain/unique_imgs/'
landmark_json_path = '/home/mohanty/Documents/ForTrain/unique_lmk_mrks/'
target_dataset_path = '/home/mohanty/VSCODE_PROJECTS/Medical-Detection3d-Toolkit-bk/detection3d/data/dataset_lmk_33/'

In [12]:
df_database = pd.read_csv("/home/mohanty/Downloads/cbct_database_15-10-2025 - database.csv")

In [15]:
img_list = [lmk_file_name.replace(".mrk.json", "") for lmk_file_name in os.listdir(landmark_json_path)]

In [16]:
img_list

['5c4ee1789155272358de84ec98719dadd5d41104026cad5dc056a8bf963910e7',
 '01544bd0c1a55d6ebb33d03cd98d0cff652f4cdf8c91b24d74f643842d0cfc69',
 '180eeaa371c030dac8a8b40b611e147af94eed00cd3a0ad490fb670b960dd794',
 '445ff7215f12f7eda33a121944ea5804c26ae476337c513924606769f3563525',
 '5c5bf2dad53005c71e1c77aa084e46f7a2f209fbe520b42c8bdba2ef24056886',
 '4600ce6789bba27af2896184c2727ecc07022d4e4895b5e3b1914478b8d4eadd',
 '4a8249fce12cc629f6fdd2cf5f685d21b6b80bc7afe5209967f52acf9fe59322',
 '18290b92c0556c25d1c4a078d861de9984676e6671caafaa190527d8c9f4be5f',
 '4f873d42cf0870a78358f7763253c8e9691b298405dfdf0349eab04f27ed7d7b',
 'f0009_0000',
 '6991a823f8032340b471780f746a2f95aa82e5948a17ebf09a050632278fbbc1',
 '32f6daa38297405b1cf3dfc9ad5a2790835adaa0d08fcba11938ccfeba160b7b',
 '0951d48dad3ff760722c2db87886e8ce6aa5a7591004d35ccb641f2d4142f1fe',
 '26f730dc1311c413349dfb8cca54e045b8a12a2f3056acee9e060b8ef8e49672',
 '005e303301cf13c59e28ca45101da37537a7feff05866111960554e3918dbd20',
 '25360a1d3e33e1d72

In [18]:
df_database.head()

,image_id,series_uid,acquisition_year,modality,institution_address,patient_name,patient_sex,image_position,image_orientation,pixel_spacing_tag,...,sent_for_annotations,lmk_annotation_batch,sent_for_lmk_annotation,s3_uri,s3_error,has_filename_match,legacy_image_id,Duplicate to,Seg_done,Seg_Review_Assigned to
0,0000d6a97dba8f56c8f81c23d08baf43dfaf75c6e6aa27...,2.16.840.114421.82324.9635618832.9667154832,2020,CT,NewTaipeiCity,c99247adf6a499efd0be567b5f085778fcc1407b893e55...,O,[ -320 -320 -31.75],[ 1 0 0 ...,[ 0.25 0.25],...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN
1,000e7d890baf53086fc97edde37b8da50be74ffa0f861a...,2.16.840.114421.82324.9656246163.9687782163,2020,CT,NewTaipeiCity,63e6d720649df88d981250371cca4932796541682dc06d...,O,[ -384 -384 -72.5],[ 1 0 0 ...,[ 0.3 0.3],...,NaN,3.0,1.0,NaN,NaN,False,NaN,NaN,NaN,NaN
2,0011d066afc499a145371fe8385a8186e9b0cd23d5c837...,2.16.840.114421.82324.9723035040.9754571040,2022,CT,NewTaipeiCity,fd27dbd65dc659ed3ee74a0267ed20ad35118193e32138...,O,[ -320 -320 -31.75],[ 1 0 0 ...,[ 0.25 0.25],...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN
3,0027a806fb6a921d9afc20774aec3b030e3ad963c8f643...,2.16.840.114421.82324.9737666852.9769202852,2023,CT,NewTaipeiCity,de4ba90ddbeb52e5944f42551c07cd3d44191e4bf713c0...,O,[ -320 -320 -31.75],[ 1 0 0 ...,[ 0.25 0.25],...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN
4,0041ae7f19b6c9d924a1cc468cf9bca755f458fe536771...,2.16.840.114421.82324.9553123740.9584659740,2017,CT,NewTaipeiCity,406fc7a63806d3736e8b73ce8a6416fc7477b1987a4568...,O,[ -384 -384 -72.5],[ 1 0 0 ...,[ 0.3 0.3],...,NaN,3.0,1.0,NaN,NaN,False,NaN,NaN,NaN,NaN


In [19]:
df_filtered = df_database[df_database["image_id"].isin(img_list)]

In [1]:
import numpy as np

def count_positive_voxels(pos_upper_bound: int):
    r = pos_upper_bound
    count = 0
    coords = []

    for dx in range(-r, r + 1):
        for dy in range(-r, r + 1):
            for dz in range(-r, r + 1):
                if np.linalg.norm([dx, dy, dz]) < r:
                    count += 1
                    coords.append((dx, dy, dz))

    return count, coords


# Example:
for r in range(2, 9):
    c, _ = count_positive_voxels(r)
    print(f"pos_upper_bound={r} -> positive voxels = {c}")


pos_upper_bound=2 -> positive voxels = 27
pos_upper_bound=3 -> positive voxels = 93
pos_upper_bound=4 -> positive voxels = 251
pos_upper_bound=5 -> positive voxels = 485
pos_upper_bound=6 -> positive voxels = 895
pos_upper_bound=7 -> positive voxels = 1365
pos_upper_bound=8 -> positive voxels = 2103


In [27]:
df_filtered['nifti_path']

1       /home/mohanty/CBCT_Processed/DentalClinic/dent...
6       /home/mohanty/CBCT_Processed/DentalClinic/dent...
8       /home/mohanty/CBCT_Processed/DentalClinic/dent...
10      /home/mohanty/CBCT_Processed/DentalClinic/dent...
25      /home/mohanty/CBCT_Processed/DentalClinic/dent...
                              ...                        
3793    /home/mohanty/CBCT_Processed/dental_batch2/ima...
3798    /home/mohanty/CBCT_Processed/dental_batch2/ima...
3807    /home/mohanty/CBCT_Processed/DentalClinic/dent...
3984    /home/mohanty/CBCT_Processed/DentalClinic/dent...
4013    /home/mohanty/CBCT_Processed/DentalClinic/dent...
Name: nifti_path, Length: 258, dtype: object

In [34]:
from pathlib import Path
import shutil

dst_dir = Path(img_volume_path)
dst_dir.mkdir(parents=True, exist_ok=True)

paths = df_filtered["nifti_path"].dropna().astype(str).tolist()

for p in paths:
    src = Path(p)
    if not src.exists():
        print(f"[SKIP missing] {src}")
        continue

    # Copy file -> destination folder (preserves filename)
    shutil.copy2(src, dst_dir / src.name)


In [35]:
import os
import json
import numpy as np
import pandas as pd
import SimpleITK as sitk
from tqdm import tqdm

from detection3d.utils.image_tools import resample_spacing
from detection3d.scripts.gen_landmark_mask import gen_single_landmark_mask


# -----------------------------------------------------------------------------
# CONFIG
# -----------------------------------------------------------------------------
img_volume_path = "/home/mohanty/Documents/ForTrain/unique_imgs/"
landmark_json_path = "/home/mohanty/Documents/ForTrain/unique_lmk_mrks/"
target_dataset_path = "/home/mohanty/VSCODE_PROJECTS/Medical-Detection3d-Toolkit-bk/detection3d/data/dataset_lmk_33/"

landmark_label = {
    "Or-Left": 1, "Or-Right": 2, "Po-Left": 3, "Po-Right": 4,
    "Nasion": 5, "S": 6, "Ba": 7, "ANS": 8, "A": 9, "PNS": 10,
    "CI": 11, "J-Left": 12, "J-Right": 13, "J_PVA": 14,
    "B": 15, "Pog": 16, "Me": 17, "Gn": 18,
    "Go_Inf-Left": 19, "Go_Inf-Right": 20, "Go_Sup-Left": 21, "Go_Sup-Right": 22,
    "Co_Pos-Left": 23, "Co_Pos-Right": 24, "Co_Sup-Left": 25, "Co_Sup-Right": 26,
    "H": 27, "Sn": 28, "Cm": 29, "Ls": 30,
}

pos_upper_bound = 3
neg_lower_bound = 6

target_image_spacing = (0.5, 0.5, 0.5)
target_image_origin = (0.0, 0.0, 0.0)

# If True, skip cases missing any landmark in landmark_label
STRICT_REQUIRE_ALL = False


# -----------------------------------------------------------------------------
# UTILS
# -----------------------------------------------------------------------------
def spacing_differs(a, b, eps=1e-6):
    """Robust spacing compare with tolerance."""
    return any(abs(x - y) > eps for x, y in zip(a, b))


def load_and_shift_landmarks_physical(json_path, shift_by_xyz, label_map):
    """
    Loads Slicer mrk.json control points (physical coords, mm).

    Since we enforce image origin -> (0,0,0), we must shift landmark positions:
      pos_new = pos_old - shift_by_xyz
    where shift_by_xyz is the old origin before we zero it.

    Returns:
      df_phys: columns [name, x, y, z]
      unknown: set of labels in json but not in label_map
    """
    with open(json_path, "r") as f:
        data = json.load(f)

    cps = data["markups"][0]["controlPoints"]
    shift = np.array(shift_by_xyz, dtype=np.float64)

    rows = []
    unknown = set()

    for cp in cps:
        name = cp["label"]
        if name not in label_map:
            unknown.add(name)
            continue

        pos = np.array(cp["position"], dtype=np.float64)
        pos = pos - shift  # ✅ shift to new origin=0 coordinate frame

        rows.append({
            "name": name,
            "x": float(np.round(pos[0], 2)),
            "y": float(np.round(pos[1], 2)),
            "z": float(np.round(pos[2], 2)),
        })

    return pd.DataFrame(rows), unknown


# -----------------------------------------------------------------------------
# MAIN
# -----------------------------------------------------------------------------
os.makedirs(target_dataset_path, exist_ok=True)

json_files = [f for f in os.listdir(landmark_json_path) if f.endswith(".json")]
print(f"[INFO] Found {len(json_files)} landmark json files.")

required_labels = set(landmark_label.keys())

for filename in tqdm(json_files, desc="Processing"):
    patient_id = filename.replace(".mrk.json", "").replace(".json", "")

    json_path = os.path.join(landmark_json_path, filename)
    img_path = os.path.join(img_volume_path, f"{patient_id}.nii.gz")

    if not os.path.exists(img_path):
        print(f"[SKIP] {patient_id}: volume not found")
        continue

    out_dir = os.path.join(target_dataset_path, patient_id)
    os.makedirs(out_dir, exist_ok=True)

    try:
        # ---------------------------------------------------------------------
        # 1) Load image (keep original origin/direction for resampling)
        # ---------------------------------------------------------------------
        image = sitk.ReadImage(img_path)

        # ---------------------------------------------------------------------
        # 2) Resample FIRST using your util (do NOT set origin before resample)
        # ---------------------------------------------------------------------
        if spacing_differs(image.GetSpacing(), target_image_spacing):
            image = resample_spacing(
                image,
                target_image_spacing,
                1,
                "LINEAR"
            )

        # After resample, record current origin (this is where landmarks live)
        old_origin = image.GetOrigin()

        # ---------------------------------------------------------------------
        # 3) Now enforce origin=0 (metadata change only)
        # ---------------------------------------------------------------------
        image.SetOrigin(target_image_origin)
        spacing = image.GetSpacing()

        # ---------------------------------------------------------------------
        # 4) Load + shift landmarks into origin=0 frame
        # ---------------------------------------------------------------------
        landmark_cos_df, unknown = load_and_shift_landmarks_physical(
            json_path=json_path,
            shift_by_xyz=old_origin,
            label_map=landmark_label
        )

        if landmark_cos_df.empty:
            print(f"[SKIP] {patient_id}: none of the required landmarks present")
            continue

        # strict mode: require all landmarks
        if STRICT_REQUIRE_ALL:
            present = set(landmark_cos_df["name"].tolist())
            missing = required_labels - present
            if missing:
                print(f"[SKIP] {patient_id}: missing required landmarks {sorted(missing)}")
                continue

        # ---------------------------------------------------------------------
        # 5) Save landmark CSV (physical coords in origin=0 frame)
        # ---------------------------------------------------------------------
        csv_out = os.path.join(out_dir, f"{patient_id}.csv")
        landmark_cos_df.to_csv(csv_out, index=False)

        # ---------------------------------------------------------------------
        # 6) Generate mask using YOUR function exactly
        # ---------------------------------------------------------------------
        landmark_mask = gen_single_landmark_mask(
            image,
            landmark_cos_df,
            landmark_label,
            spacing,
            pos_upper_bound,
            neg_lower_bound
        )

        # Make sure mask metadata matches image (origin=0, spacing, direction)
        landmark_mask.CopyInformation(image)

        # ---------------------------------------------------------------------
        # 7) Save outputs
        # ---------------------------------------------------------------------
        img_out = os.path.join(out_dir, f"{patient_id}.nii.gz")
        mask_out = os.path.join(out_dir, f"{patient_id}_landmark_mask.nii.gz")

        sitk.WriteImage(image, img_out, useCompression=True)
        sitk.WriteImage(landmark_mask, mask_out, useCompression=True)

    except Exception as e:
        print(f"[ERROR] {patient_id}: {e}")


[INFO] Found 288 landmark json files.


Processing: 100%|██████████| 288/288 [17:02<00:00,  3.55s/it]


In [18]:
import os

def delete_files_with_extensions(directory, extensions):
    """
    Walk through the given directory and its subdirectories, find files with specified extensions,
    print their names, and delete them.
    
    :param directory: The root directory to start the search.
    :param extensions: A tuple of file extensions to look for (e.g., ('.jpg', '.png')).
    """
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(extensions):
                file_path = os.path.join(root, file)
                # print(file_path)
                print(f"Deleting file: {file_path}")
                os.remove(file_path)

# Example usage
directory_to_search = img_volume_path  # Change this to your target directory
file_extensions = '.identifier'  # Add or modify extensions as needed

delete_files_with_extensions(directory_to_search, file_extensions)


# Dataset Structure

The dataset is organized in the following folder structure:


In [19]:
# data_set
# ├── patient_id1
# │   ├── patient_id1_landmark_mask.nii.gz
# │   ├── patient_id1.csv
# │   └── patient_id1.nii.gz
# ├── patient_id2
# │   ├── patient_id2_landmark_mask.nii.gz
# │   ├── patient_id2.csv
# │   └── patient_id2.nii.gz
# ├── patient_id3
# ├── patient_id..
# ├── patient_idn

Generating train and test .csv file

In [56]:
import os

rows = []

for dirpath, dirnames, filenames in os.walk(target_dataset_path):
    if not filenames:
        continue

    image_path = None
    landmark_mask_path = None
    landmark_file_path = None

    for filename in filenames:
        fp = os.path.join(dirpath, filename)

        if filename.endswith(".nii.gz"):
            if filename.endswith("mask.nii.gz"):
                landmark_mask_path = fp
            else:
                image_path = fp

        elif filename.endswith(".csv"):
            landmark_file_path = fp

    if os.path.basename(dirpath) !='':
        rows.append({
            "image_name": os.path.basename(dirpath),
            "image_path": image_path,
            "landmark_file_path": landmark_file_path,
            "landmark_mask_path": landmark_mask_path,
        })


In [57]:
len(rows)

288

In [58]:
data_df = pd.DataFrame(rows)

In [59]:
data_df

,image_name,image_path,landmark_file_path,landmark_mask_path
0,44cd3e905a18fbe8a77687e2634c1a91b92bad94f5d28b...,/home/mohanty/VSCODE_PROJECTS/Medical-Detectio...,/home/mohanty/VSCODE_PROJECTS/Medical-Detectio...,/home/mohanty/VSCODE_PROJECTS/Medical-Detectio...
1,0808e44943b01c0838d0108ebf1f8516e3e600afc03c2b...,/home/mohanty/VSCODE_PROJECTS/Medical-Detectio...,/home/mohanty/VSCODE_PROJECTS/Medical-Detectio...,/home/mohanty/VSCODE_PROJECTS/Medical-Detectio...
2,15da167cbf9eba9993dc68153306ea6333165ba3fd7171...,/home/mohanty/VSCODE_PROJECTS/Medical-Detectio...,/home/mohanty/VSCODE_PROJECTS/Medical-Detectio...,/home/mohanty/VSCODE_PROJECTS/Medical-Detectio...
3,f0016_0000,/home/mohanty/VSCODE_PROJECTS/Medical-Detectio...,/home/mohanty/VSCODE_PROJECTS/Medical-Detectio...,/home/mohanty/VSCODE_PROJECTS/Medical-Detectio...
4,b6d720db7f86a0db64577ea74d6523c99a01262f2ffd5f...,/home/mohanty/VSCODE_PROJECTS/Medical-Detectio...,/home/mohanty/VSCODE_PROJECTS/Medical-Detectio...,/home/mohanty/VSCODE_PROJECTS/Medical-Detectio...
...,...,...,...,...
283,066b2c19aae435ebf1e5bd9b1ec6a4166bb9b02e0e49a4...,/home/mohanty/VSCODE_PROJECTS/Medical-Detectio...,/home/mohanty/VSCODE_PROJECTS/Medical-Detectio...,/home/mohanty/VSCODE_PROJECTS/Medical-Detectio...
284,55b35ea7218466000a944b95dcf67294bc1656f1b76aca...,/home/mohanty/VSCODE_PROJECTS/Medical-Detectio...,/home/mohanty/VSCODE_PROJECTS/Medical-Detectio...,/home/mohanty/VSCODE_PROJECTS/Medical-Detectio...
285,1f49e01fca49894487784c13c98dd93f8f131129522dfe...,/home/mohanty/VSCODE_PROJECTS/Medical-Detectio...,/home/mohanty/VSCODE_PROJECTS/Medical-Detectio...,/home/mohanty/VSCODE_PROJECTS/Medical-Detectio...
286,690aa8f722bd75cf39f6ee5ff38fea0ac96c92c9d00be5...,/home/mohanty/VSCODE_PROJECTS/Medical-Detectio...,/home/mohanty/VSCODE_PROJECTS/Medical-Detectio...,/home/mohanty/VSCODE_PROJECTS/Medical-Detectio...


In [60]:
from sklearn.model_selection import KFold

# Initialize KFold with 3 splits
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Enumerate over the KFold splits and save to files
for fold, (train_index, val_index) in enumerate(kf.split(data_df), 1):
    train_set = data_df.iloc[train_index]
    val_set = data_df.iloc[val_index]
    
    # Save to CSV files
    train_set.to_csv(os.path.join(target_dataset_path, f'train_fold{fold}.csv'), index=False)
    val_set.to_csv(os.path.join(target_dataset_path, f'val_fold{fold}.csv'), index=False)

print("Files saved successfully!")

Files saved successfully!
